Array Storage Order
===================
In Python (and C), n-dimensional arrays are typically stored in row-major order, whereas `Fortran` or `Matlab` default to column-major ordering.  `mrrt.mri` uses the Fortran (column-major) ordering convention to better match other similar MRI toolboxes such as Gadgetron, the Berkley Advanced Reconstruction Toolbox (BART), and the Michigan Image Reconstruction Toolbox (MIRT).

For an array stored in contiguous memory, column major order means that the elements along the first dimension of the array are adjacent in memory while those along the last axis are the farthest appart.  This is particularly important to remember when reshaping arrays.  Functions such as `ravel` and `reshape` in the `NumPy` package assume C (row-major) ordering by default.  To get the desired Fortran ordering, one must pass `order='F'` to these functions.

So for the following 2D matrix of shape (3, 2) (axis 0 is displayed vertically):
$$a = \begin{bmatrix} 1 & 4 \\ 2 & 5 \\ 3 & 6 \end{bmatrix}$$

Calling `numpy.ravel(a, order='F')` would create the following 1D vector.
$$\begin{bmatrix} 1 & 2 & 3 & 4 & 5 & 6 \end{bmatrix}$$
(note that the output of ravel will have a shape that is (6, ) so it is not specifically a column or row vector)

If the array was originally created with Fortran ordering, this also corresponds to the order of the elements of the original array in memory.

Calling `numpy.ravel(a, order='C')` would create the following 1D vector.
$$\begin{bmatrix} 1 & 4 & 2 & 5 & 3 & 6 \end{bmatrix}$$

An example illustrating array attributes revealing the ordering and strides of a numpy array is given below.

In [2]:
# small example
import numpy as np
a = np.asarray([[1, 4], [2, 5], [3, 6]], order='F')
print("a =")
print(a)
print('a.shape = {}\n'.format(a.shape))
print('a.flags:')
print(a.flags)

a_raveled = a.ravel(order='F')
print("\n\na = {}".format(a_raveled))
print('a_raveled.flags:')
print(a_raveled.flags)  # Note:  a 1D array is both C and Fortan-contiguous

# size of 1 element of the array in bytes
print("\n\nDTYPE & STRIDE INFO")
print("a.dtype = {}".format(a.dtype))
print("size of 1 element = {} bytes".format(a.itemsize))
print("shape = {}".format(a.shape))
print("shape = {}".format(a.shape))
print("strides = {} bytes".format(a.strides))
print("strides = {} elements".format(tuple(np.asarray(a.strides)//a.itemsize)))
print("Fortran ordering -> strides are smallest along first dimension")

# now convert to C-ordering
a = np.ascontiguousarray(a)
print("\n\nC-contiguous STRIDE INFO")
print("strides (C-order) = {} bytes".format(a.strides))
print("strides (C-order) = {} elements".format(tuple(np.asarray(a.strides)//a.itemsize)))
print("c ordering -> strides are largest along first dimension")


a =
[[1 4]
 [2 5]
 [3 6]]
a.shape = (3, 2)

a.flags:
  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False


a = [1 2 3 4 5 6]
a_raveled.flags:
  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False


DTYPE & STRIDE INFO
a.dtype = int64
size of 1 element = 8 bytes
shape = (3, 2)
shape = (3, 2)
strides = (8, 24) bytes
strides = (1, 3) elements
Fortran ordering -> strides are smallest along first dimension


C-contiguous STRIDE INFO
strides (C-order) = (16, 8) bytes
strides (C-order) = (2, 1) elements
c ordering -> strides are largest along first dimension


Summary of some related numpy functions
=======================================

convert between memory layouts
------------------------------

- **np.ascontiguousarray** | convert array to C order 
- **np.asfortranarray**    | convert array to F order


reshaping
---------

- **np.reshape**           | reshape an array
- **np.ravel**             | ravel an nd-array to 1D
- **np.ravel_multi_index** | convert from nd-indices to a 1D flat index
- **np.unravel_index**     | convert from a 1D flat index to a nd-index

combining arrays
----------------

- **np.concatenate** | concatenate arrays along any existing axis
- **np.hstack**      | stack a sequence of arrays along the first axis
- **np.vstack**      | stack  a sequence of arrays along the second axis
- **np.stack**       | stack a sequence of n-dimensional arrays into an (n+1)-dim. array


Conventions for MR k-space data dimensions
=====================================

3D images are assumed to be ordered as (x, y, z).  When there are multiple coils and dynamics, ordering would be

`(x, y, z, coils, dynamics)`

For the 2D case this would be just:

`(x, y, coils, dynamics)`

Here 'dynamics' could be any number of additional dimensions corresponding to echos, segments, repetitions, etc.

The linear operators in `mrrt.mri` will automatically reshape data as needed. If the data is already Fortran-ordered, this will typically not involve making a copy.

k-space trajectory dimensions and units
================================
kspace coordinates should be stored in a (Fortran-ordered) 2d array of shape (nsamples, ndim) where e.g. `ndim = 2` for 2D and `ndim = 3` for 3D

if the field-of-view (FOV) is specified in mm then the kspace should be in units 1/mm
The use of physical units for the k-space is consistent with the Michigan Image Reconstruction Toolbox, but differs from some other packages which use k-space normalized to the range [-0.5, 0.5] or [-N/2, N/2].

Conventions for coil sensitivity map dimensions
===============================================
Assume a 4d map of coil sensitivities of shape (nx, ny, nz, ncoils).

This should be reshaped into a 2D Fortran-ordered array of shape (nx*ny*nz, ncoils). This can be done as

```Python
coil_sensitivties = cmap4d.reshape((-1, ncoils), order='F')
```